<a href="https://colab.research.google.com/github/lphohmann/DL_microbial_gene_classifier/blob/main/build_datablock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
# do i need this still? 
#!pip install fastai --upgrade

In [1]:
#hide
!pip install torchtext==0.8.1
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 7.0MB 7.6MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 727kB 7.3MB/s 
     |████████████████████████████████| 1.2MB 35.9MB/s 
     |████████████████████████████████| 204kB 52.5MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
Mounted at /content/gdrive


In [2]:
#hide
import fastai
from fastbook import *

In [3]:
import numpy as np

In [4]:
# mount google drive to access files and set the correct working
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/ColabNotebooks/DL_project/
!pwd

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/DL_project
/content/drive/MyDrive/ColabNotebooks/DL_project


In [11]:
# cell to check function documentation
??nn.Conv2d

In [5]:
# read in my data from which training and validation set will be created
trainval = pd.read_csv('trainval.csv', low_memory=False)

In [6]:
# defining functions required for building the DataBlock
# one hot encoding function
from sklearn.preprocessing import OneHotEncoder
def OH_enc(seq: str):
    # get the categories into array
    cats = ['K', 'D', 'N', 'E', 'R', 'A', 'T', 'L', 'I', 'Q', 'C', 'F', 'G', 'W', 'M', 'S', 'H', 'P', 'V', 'Y']
    cat_array = np.array(sorted(cats), ndmin=1) #
    # get seq into array
    seq_array = np.array(list(seq))
    #one hot encode the sequence
    onehot_encoder = OneHotEncoder(categories=[cat_array],sparse=False,handle_unknown='ignore')
    onehot_encoded_seq = onehot_encoder.fit_transform(seq_array.reshape(len(seq_array), 1))
    return np.transpose(onehot_encoded_seq)

# zero padding function
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)
    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

# combine in one function
def main_item_tfms(seq): # added -> None because the tensor shape later is 1,2820 and i think it should be 20,2820
    enc_seq = OH_enc(seq)
    pad_encseq_array = pad_along_axis(enc_seq,2820,1)
    return pad_encseq_array 

# categorize function
tcat = Categorize(vocab=['K00024', 'K00121', 'K00122', 'K00123', 'K00124', 'K00126',
       'K00127', 'K00148', 'K00169', 'K00170', 'K00171', 'K00172',
       'K00194', 'K00196', 'K00197', 'K00198', 'K00200', 'K00201',
       'K00202', 'K00317', 'K00320', 'K00441', 'K00600', 'K00625',
       'K00672', 'K00830', 'K00925', 'K01007', 'K01070', 'K01499',
       'K01595', 'K01895', 'K03388', 'K03389', 'K03390', 'K05979',
       'K06034', 'K08097', 'K08691', 'K08692', 'K09733', 'K10713',
       'K10714', 'K11212', 'K11261', 'K11779', 'K11780', 'K12234',
       'K13039', 'K13788', 'K14067', 'K14080', 'K14083', 'K14940',
       'K14941', 'K15228', 'K15229', 'K18277'])

# questoin, do i have to also apply the ToTensor()? https://docs.fast.ai/tutorial.pets.html
# -> answer : I think ToTensor() is always applied, like to float tensor
tconv = ToTensor()
# try to add this as well (add a batch_tfms (or after_batch_tfms) line to DataBlock)
#batch_tfms = [IntToFloatTensor(), Normalize.from_stats(mean,std)]
tofloat = IntToFloatTensor()

# get_x and y by specifying column in dataframe
def get_y(r): return tcat(r['Knum']) # apply the categorize trasnform

# maybe add another dimension here, which is the number of channels (later needed)
def get_x(r): # apply the one hot encoding and padding function
    return np.expand_dims(main_item_tfms(r['Seq']),0) # this should add a dimension at 0

# building the datablock
dblock = DataBlock(
                 splitter = TrainTestSplitter(test_size=0.2, random_state=42, stratify=trainval[['Knum']]),
                 get_x = get_x,
                 get_y = get_y,
                 item_tfms = tconv,
                 batch_tfms = tofloat
                 )

# checking it
dblock.summary(trainval)

Setting-up type transforms pipelines
0           176716  K00600   
1           240411  K00925   
2             3713  K00024   
3           257392  K01070   
4           237982  K00925   
...            ...     ...   
479484      277328  K01595   
479485      117389  K00127   
479486      108061  K00124   
479487      306941  K01895   
479488       69614  K00123   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            Seq  
0                     

In [7]:
# create dataloaders from datablock 
dls = dblock.dataloaders(trainval,bs=256) # not sure how to choose the batch size

In [8]:
# check one batch to make sure the dls is constructed right
x,y = dls.one_batch()
#y # these should be the targets 
#x # and these the encoded sequences

# check the shape
x.shape, y.shape # torch.Size([256, 1, 20, 2820]), batch size,channel,height, width 
#y.shape
#  Conv2d layers expect input with the shape (n_samples, channels, height, width) 
# e.g., (1000, 1, 224, 224) so  add a channel dimension
# tensor = tensor.unsqueeze(1)  # adds a dimensions at the specified index.



(torch.Size([256, 1, 20, 2820]), torch.Size([256]))

change the kernels.
way to check the dimensions of the tensor
i think its 20x2820 (set to 20 kernels now). Also what should be the input for the output in each layer (20x20 to how many output nodes)


In [ ]:
# model 2
from fastai.callback.hook import *
# conv function
def conv(ni, nf, ks=20, act=True):
    # input - input tensor of shape (minibatch, in_channels, iH, iW) height and width of the image 
    # weight - filters of shape (out_channels, in_channels, kH, kW) height and width of our kernel
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2) 
    if act: res = nn.Sequential(res, nn.ReLU())
    return res
# the model
def simple_cnn(): 
    return sequential(
        conv(1 ,8, ks=5), #14x14 
        conv(8 ,16), #7x7 
        conv(16,32), #4x4 
        conv(32,64), #2x2 
        conv(64,58, act=False), #1x1 # 58 would correstpond to the number of categories
        Flatten(),
    )
# fit
def fit(epochs=1, lr=0.06):
    learn = Learner(dls, simple_cnn(), loss_func=F.cross_entropy,
                        metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit_one_cycle(epochs, lr) # make training more stable with fit_one_cycle
    return learn

# checing things
simple_cnn(xb).shape # 2nd field shoudl correspond to number of possible outcomes
# fit the model
learn = fit()
learn.summary()
# checking activation
learn.activation_stats.plot_layer_stats(-2) # check activations at the end, shouldnt be near zero
learn.activation_stats.color_dim(-4)
# view learning rate
learn.recorder.plot_sched()

In [9]:
# model 3
# alt. cov function with batchnorm layer 
def conv(ni, nf, ks=21, act=True):
    layers = [nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)] # stride 2 -> kernel applied to every second 'pixel'
    layers.append(nn.BatchNorm2d(nf))
    if act: layers.append(nn.ReLU())
    return nn.Sequential(*layers)
# the model
def simple_cnn(): 
    return sequential(
        conv(1 ,200, ks=21), # kernel of 21x21, to 200 filters
        conv(8 ,16), #7x7 
        conv(16,32), #4x4 
        conv(32,64), #2x2 
        conv(64,58, act=False), #1x1 
        Flatten(),
    )
# fit
def fit(epochs=1, lr=0.06):
    learn = Learner(dls, simple_cnn(), loss_func=F.cross_entropy,
                        metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit_one_cycle(epochs, lr) # make training more stable with fit_one_cycle
    return learn
# fit the model
learn = fit()


/usr/local/lib/python3.7/dist-packages/fastai/callback/core.py:50: UserWarning: You are shadowing an attribute (modules) that exists in the learner. Use `self.learn.modules` to avoid this
  warn(f"You are shadowing an attribute ({name}) that exists in the learner. Use `self.learn.{name}` to avoid this")


epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

In [ ]:
# model 3 checking
# checking activation
learn.activation_stats.plot_layer_stats(-2) # check activations at the end, shouldnt be near zero
learn.activation_stats.color_dim(-4)
# view learning rate
learn.recorder.plot_sched()

In [ ]:
# train for more epochs
learn = fit(5, lr=0.1)

In [ ]:
# check what mistakes the model is making (classifying one category mistakenly as another one)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()